In [20]:
import os
import pandas as pd
from pathlib import Path
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'


In [21]:
# acquire system path of the current file, join it with the related path of the csv file path.
increase_path = "../Data/Increase/stocks-increase-change-in-open-interest-11-13-2023.csv"
decrease_path = "../Data/Decrease/stocks-decrease-change-in-open-interest-11-13-2023.csv"

#read csv files
increase = pd.read_csv(Path(increase_path))
decrease = pd.read_csv(Path(decrease_path))

print("Increase DataFrame column types:")
print(increase.dtypes)

print("\nDecrease DataFrame column types:")
print(decrease.dtypes)



Increase DataFrame column types:
Symbol       object
Price~      float64
Type         object
Strike       object
Exp Date     object
DTE         float64
Bid         float64
Ask         float64
Last        float64
Volume      float64
Open Int    float64
OI Chg       object
Delta       float64
IV           object
Time         object
dtype: object

Decrease DataFrame column types:
Symbol       object
Price~      float64
Type         object
Strike       object
Exp Date     object
DTE         float64
Bid         float64
Ask         float64
Last        float64
Volume      float64
Open Int    float64
OI Chg       object
Delta       float64
IV           object
Time         object
dtype: object


In [22]:
# Convert 'Strike' column to float64 in the Decrease DataFrame
increase[['OI Chg', 'Strike']] = increase[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)
decrease[['OI Chg', 'Strike']] = decrease[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)

#combine increase and decrease data
combine_df = increase.merge(decrease, how= 'outer')

DTE = "max"
#Select the DTE
if DTE == 'min':
    combine_min_DTE = combine_df[combine_df['DTE'] == combine_df['DTE'].min()]
elif DTE == 'max':
    combine_min_DTE = combine_df[combine_df['DTE'] <= combine_df['DTE'].max()]
else:
    combine_min_DTE = combine_df[combine_df['DTE'] <= DTE]

#Clean the NA data
combine_min_DTE = combine_min_DTE.dropna()

# Transfer columns 'OI Chg' and 'Strike' datatype from str to float.
combine_min_DTE[['OI Chg', 'Strike']] = combine_min_DTE[['OI Chg', 'Strike']].replace({',': '', 'unch': '0', r'\*': ''}, regex=True).astype(float)

#Transfer column 'IV'datatype from str to float.
combine_min_DTE['IV'] = combine_min_DTE['IV'].str.replace('%', '').str.replace(',', '').astype(float)/100

#Sorted by orders 'Symbol','Type','Strike','Open Int','Volume','OI Chg','IV'.
combine_sort_df = combine_min_DTE.sort_values(['Symbol','Type','Strike','Open Int','Volume','OI Chg','IV'])
combine_sort_df.columns = ['Symbol','Price','Type','Strike','Exp Date','DTE','Bid','Midpoint','Ask','Last','Volume','Open Int','OI Chg','IV','Time']

# return data
combine_sort_df

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,OI Chg,IV,Time
1037,AA,24.64,Call,25.0,2023-12-15,32.0,1.29,1.30,1.29,5391.0,5167.0,371.0,0.506936,0.4700,15:15 ET
1226,AA,24.64,Call,25.0,2023-11-17,4.0,0.42,0.44,0.47,1453.0,7181.0,267.0,0.438329,0.5258,14:49 ET
2385,AA,24.64,Call,30.0,2023-12-15,32.0,0.18,0.20,0.19,2098.0,8197.0,-7.0,0.113543,0.4992,14:59 ET
1063,AAL,11.79,Call,11.0,2023-12-15,32.0,1.03,1.06,1.03,854.0,4868.0,351.0,0.735103,0.4092,15:09 ET
2820,AAL,11.79,Call,11.0,2024-01-19,67.0,1.31,1.33,1.33,1271.0,6058.0,0.0,0.676142,0.4495,15:15 ET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,ZM,61.63,Put,130.0,2024-01-19,67.0,68.40,68.60,68.45,22720.0,2205.0,0.0,-0.960309,1.0070,15:17 ET
760,ZS,177.78,Call,180.0,2023-11-17,4.0,3.15,3.30,3.33,773.0,1681.0,580.0,0.437893,0.5498,15:11 ET
2581,ZS,177.80,Call,210.0,2024-07-19,249.0,19.35,19.65,19.00,500.0,1350.0,0.0,0.452180,0.4845,11:53 ET
2546,ZS,177.80,Put,145.0,2024-01-19,67.0,3.60,3.75,3.65,765.0,2270.0,-1.0,-0.151462,0.5469,13:09 ET


In [23]:
def connect_data_base(database=SQL_DB, user=SQL_USER, password=SQL_PASSWORD, host=SQL_HOST):
    con = psycopg2.connect(database=SQL_DB, user=SQL_USER, password=SQL_PASSWORD, host=SQL_HOST)
    cur = con.cursor()
    cur.execute('SELECT version()')
    version = cur.fetchone()[0]
    print(f"DataBaseFlow Connected!\nVersion: {version}")
    return con, cur

import sys
import os
import psycopg2
import psycopg2.extras
import pandas as pd
import pytz
from datetime import datetime
import traceback

# Load and assign variables.

SQL_DB = 'fomostop'
SQL_USER = 'postgres'
SQL_PASSWORD = 'l5F|XSfOdY]rYdZ?'
SQL_HOST = '192.168.123.88'

date = "11-13-2023"

now_timestamp = int(datetime.now().timestamp()) #时间戳只保留秒（去掉更低的精度，保留整数部分）
date_parts = date.split('-')  #分割日期字符串
con, cur = connect_data_base() #连接数据库



DataBaseFlow Connected!
Version: PostgreSQL 14.8 on x86_64-pc-linux-musl, compiled by gcc (Alpine 12.2.1_git20220924-r10) 12.2.1 20220924, 64-bit


In [24]:
combine_data = combine_sort_df

# 自定义函数，根据"Last"和"Midpoint"的关系判断"sentiment"
def get_initiator(row):
    if row['Last'] < row['Bid']:
        return 'Aggressive Seller'
    elif row['Last'] > row['Ask']:
        return 'Aggressive Buyer'
    elif row['Last'] < row['Midpoint']:
        return 'Seller'
    elif row['Last'] > row['Midpoint']:
        return 'Buyer'
    else:
        return 'Neutral'
# Add "Initiator" column
combine_data['Initiator'] = combine_data.apply(get_initiator, axis=1)

#重新排列，把'Initiator'放在'Last'后：
new_order = ['Symbol', 'Price', 'Type', 'Strike', 'Exp Date', 'DTE', 'Bid', 'Midpoint', 'Ask', 'Last', 'Initiator', 'Volume', 'Open Int', 'OI Chg', 'IV', 'Time']

# 使用reindex()方法重新排列列的顺序
combine_data = combine_data.reindex(columns=new_order)

#构建写操作的表名
write_table_name = f"_{date_parts[2]}_{date_parts[0]}_{date_parts[1]}_stocks_{now_timestamp}"  # 按照'YYYY_MM_DD'格式重新排列日期部分

# Delete old data from the table if it exists
delete_query = f"DROP TABLE IF EXISTS {write_table_name}"
cur.execute(delete_query)
con.commit()

# Create table if not exists
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {write_table_name} (
    "Symbol" TEXT,
    "Price" NUMERIC,
    "Type" TEXT,
    "Strike" NUMERIC,
    "Exp Date" TEXT,
    "DTE" NUMERIC,
    "Bid" NUMERIC,
    "Midpoint" NUMERIC,
    "Ask" NUMERIC,
    "Last" NUMERIC,
    "Initiator" TEXT,
    "Volume" NUMERIC,
    "Open Int" NUMERIC,
    "OI Chg" NUMERIC,
    "IV" NUMERIC,
    "Time" TEXT
)
"""
cur.execute(create_table_query)
con.commit()

# 批量插入数据
insert_query = f"INSERT INTO {write_table_name} VALUES %s"
rows = [tuple(row) for _, row in combine_data.iterrows()]
psycopg2.extras.execute_values(cur, insert_query, rows)
con.commit()
print("Data inserted successfully!")
print(f"{write_table_name}")

Data inserted successfully!
_2023_11_13_stocks_1700002181
